In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# ✅ Safe Colab notebook cleaner for GitHub
import nbformat
from google.colab import files

# 1️⃣ Upload your notebook (choose assignment1.ipynb)
uploaded = files.upload()
notebook_name = list(uploaded.keys())[0]
print(f"Uploaded notebook: {notebook_name}")

# 2️⃣ Read notebook
with open(notebook_name, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# 3️⃣ Remove widget metadata
if "widgets" in nb["metadata"]:
    nb["metadata"].pop("widgets", None)

# 4️⃣ Clear all outputs and execution counts
for cell in nb["cells"]:
    if "outputs" in cell:
        cell["outputs"] = []
    if "execution_count" in cell:
        cell["execution_count"] = None

# 5️⃣ Save cleaned notebook for GitHub
cleaned_name = "task_01.ipynb"
with open(cleaned_name, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print(f"✅ Cleaned notebook saved as: {cleaned_name}")

# 6️⃣ Download the cleaned notebook
files.download(cleaned_name)


In [ ]:


import pandas as pd


df = pd.read_csv("archive.zip", encoding='latin1')


print("First 5 rows of dataset:")
print(df.head())


print("\nDataset Info:")
print(df.info())

print("\nSummary Statistics:")
print(df.describe())

print("\nMissing Values:")
print(df.isnull().sum())

# Convert 'Order Date' to datetime format
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

# Display the date range of orders
print("\nOrder Date Range:", df['Order Date'].min(), "to", df['Order Date'].max())

# Check unique regions/categories to understand business dimensions
print("\nUnique Regions:", df['Region'].unique())
print("Unique Categories:", df['Category'].unique())


In [ ]:


# Make sure Order Date is datetime (already converted in Step 1)
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

# Aggregate sales by month
monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M')).sum()['Sales'].reset_index()

# Rename columns for clarity
monthly_sales.columns = ['Order Date', 'Sales']

# Check first few rows
print("Monthly Aggregated Sales:")
print(monthly_sales.head())

# Check shape (number of months)
print("\nTotal Months in Dataset:", monthly_sales.shape[0])

# Check if there are missing months (Prophet/ARIMA need continuous time series)
all_months = pd.date_range(start=monthly_sales['Order Date'].min(),
                           end=monthly_sales['Order Date'].max(), freq='M')

missing_months = all_months.difference(monthly_sales['Order Date'])
print("\nMissing Months:", missing_months)


In [ ]:


import matplotlib.pyplot as plt
import seaborn as sns

# 1. Plot overall monthly sales trend
plt.figure(figsize=(12,6))
plt.plot(monthly_sales['Order Date'], monthly_sales['Sales'], marker='o')
plt.title("Monthly Sales Trend (2014 - 2017)", fontsize=16)
plt.xlabel("Date")
plt.ylabel("Sales")
plt.grid(True)
plt.show()

# 2. Add a Year and Month column for seasonality analysis
monthly_sales['Year'] = monthly_sales['Order Date'].dt.year
monthly_sales['Month'] = monthly_sales['Order Date'].dt.month

# Group by Month (ignoring year) to see average seasonal pattern
monthly_avg = monthly_sales.groupby('Month')['Sales'].mean()

plt.figure(figsize=(10,5))
sns.lineplot(x=monthly_avg.index, y=monthly_avg.values, marker='o')
plt.title("Average Sales by Month (Seasonality)", fontsize=16)
plt.xlabel("Month")
plt.ylabel("Average Sales")
plt.xticks(range(1,13))
plt.grid(True)
plt.show()

# 3. Heatmap of Year vs Month (for seasonality pattern)
pivot = monthly_sales.pivot(index="Month", columns="Year", values="Sales")

plt.figure(figsize=(10,6))
sns.heatmap(pivot, cmap="YlGnBu", annot=True, fmt=".0f")
plt.title("Monthly Sales Heatmap (2014 - 2017)", fontsize=16)
plt.xlabel("Year")
plt.ylabel("Month")
plt.show()


In [ ]:


import statsmodels.api as sm

# Ensure Order Date is datetime index
monthly_sales.set_index('Order Date', inplace=True)

# Apply seasonal decomposition (additive model)
decomposition = sm.tsa.seasonal_decompose(monthly_sales['Sales'], model='additive')

# Plot decomposition
fig = decomposition.plot()
fig.set_size_inches(12, 8)
plt.show()

# Reset index for next steps (Prophet needs columns, not index)
monthly_sales.reset_index(inplace=True)


In [ ]:
# Define the split ratio (e.g., 80% train, 20% test)
train_size = int(len(df) * 0.8)

# Split the data
train = df.iloc[:train_size]
test = df.iloc[train_size:]

# Check the sizes
print(f"Total data points: {len(df)}")
print(f"Training data points: {len(train)}")
print(f"Testing data points: {len(test)}")

# Optional: visualize the split
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(train['Sales'], label='Train')
plt.plot(test['Sales'], label='Test')
plt.title("Train-Test Split")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.show()


In [ ]:
from prophet import Prophet
import matplotlib.pyplot as plt
import pandas as pd

# -------------------------------
# Prepare data
df_prophet = df.reset_index().rename(columns={'Order Date': 'ds', 'Sales': 'y'})

# Convert to datetime and drop invalid dates
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'], errors='coerce')
df_prophet = df_prophet.dropna(subset=['ds']).drop_duplicates(subset=['ds']).sort_values('ds')

# -------------------------------
# Train-test split: 80% train, 20% test
split_index = int(len(df_prophet) * 0.8)
train_prophet = df_prophet.iloc[:split_index].copy()
test_prophet = df_prophet.iloc[split_index:].copy()

if test_prophet.empty:
    raise ValueError(f"Test set is empty! Dataset too small after cleaning dates. "
                     f"Rows in dataset: {len(df_prophet)}")

# -------------------------------
# Fit Prophet model
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
model.fit(train_prophet)

# -------------------------------
# Create future dataframe covering test period
first_test_date = test_prophet['ds'].min()
last_test_date = test_prophet['ds'].max()
future_dates = pd.date_range(start=first_test_date, end=last_test_date, freq='D')
future = pd.DataFrame({'ds': future_dates})

# Predict
forecast = model.predict(future)

# -------------------------------
# Align forecast with actual test dates
forecast_test = forecast.loc[forecast['ds'].isin(test_prophet['ds']), 'yhat'].values

# Check alignment
if len(forecast_test) != len(test_prophet):
    raise ValueError(f"Length mismatch: forecast={len(forecast_test)}, test={len(test_prophet)}")

# -------------------------------
# Plot Train, Test, Forecast
plt.figure(figsize=(12,6))
plt.plot(train_prophet['ds'], train_prophet['y'], label='Train')
plt.plot(test_prophet['ds'], test_prophet['y'], label='Test')
plt.plot(test_prophet['ds'], forecast_test, label='Forecast', color='red')
plt.title("Prophet Forecast vs Actual Sales")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# True values from test set
y_true = test_prophet['y'].values

# Predicted values from Step 6
y_pred = forecast_test

# -------------------------------
# Compute metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)

# MAPE: avoid division by zero
non_zero_idx = y_true != 0
mape = np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

# -------------------------------
# Print metrics
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")



In [ ]:

!pip install plotly ipywidgets --quiet

import plotly.graph_objects as go
from ipywidgets import interact, IntSlider
import pandas as pd

# -------------------------------
# Assume df_prophet (original data), forecast (from Prophet), test_prophet exist
# Merge forecast with test data for alignment
forecast_df = forecast[['ds', 'yhat']].merge(test_prophet[['ds','y']], on='ds', how='left')

# -------------------------------
# Function to plot forecast
def interactive_forecast(days=30):
    fig = go.Figure()
    # Actual sales
    fig.add_trace(go.Scatter(x=df_prophet['ds'], y=df_prophet['y'],
                             mode='lines', name='Actual'))
    # Forecasted sales
    fig.add_trace(go.Scatter(x=forecast_df['ds'][:days], y=forecast_df['yhat'][:days],
                             mode='lines', name='Forecast'))
    fig.update_layout(title=f'Sales Forecast (first {days} days)',
                      xaxis_title='Date', yaxis_title='Sales')
    fig.show()

# -------------------------------
# Show default chart immediately
interactive_forecast(days=30)

# -------------------------------
# Add interactive slider for forecast horizon
interact(interactive_forecast,
         days=IntSlider(value=30, min=1, max=len(forecast_df), step=1))


In [ ]:


import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from google.colab import files

# 1. Upload CSV manually
uploaded = files.upload()  # select your superstore.csv file
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name, encoding="latin1")

# 2. Prepare data
df = df[['Order Date', 'Sales']].rename(columns={'Order Date':'ds','Sales':'y'})
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values('ds')

# 3. Train Prophet model
model = Prophet(daily_seasonality=True)
model.fit(df)

# 4. Forecast future sales
future_days = 30
future = model.make_future_dataframe(periods=future_days)
forecast = model.predict(future)

# 5. Align actuals and predictions
merged = pd.merge(df, forecast[['ds','yhat']], on='ds', how='left')
y_true = merged['y'].values
y_pred = merged['yhat'].values

# 6. Calculate metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# 7. Plot actual vs forecast
plt.figure(figsize=(12,6))
plt.plot(df['ds'], df['y'], label='Actual Sales')
plt.plot(forecast['ds'], forecast['yhat'], label='Forecast', color='red')
plt.title(f"Sales Forecast for next {future_days} days")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.show()


In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from google.colab import files

# 1. Upload CSV manually
uploaded = files.upload()  # select your superstore.csv file
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name, encoding="latin1")

# 2. Prepare data
df = df[['Order Date', 'Sales']].rename(columns={'Order Date':'ds','Sales':'y'})

# ✅ Fix: parse dates with dayfirst=True (since your data is DD/MM/YYYY)
df['ds'] = pd.to_datetime(df['ds'], dayfirst=True, errors='coerce')

df = df.sort_values('ds')

# 3. Train Prophet model
model = Prophet(daily_seasonality=True)
model.fit(df)

# 4. Forecast future sales
future_days = 30
future = model.make_future_dataframe(periods=future_days)
forecast = model.predict(future)

# 5. Align actuals and predictions
merged = pd.merge(df, forecast[['ds','yhat']], on='ds', how='left')
y_true = merged['y'].values
y_pred = merged['yhat'].values

# 6. Calculate metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# 7. Plot actual vs forecast
plt.figure(figsize=(12,6))
plt.plot(df['ds'], df['y'], label='Actual Sales')
plt.plot(forecast['ds'], forecast['yhat'], label='Forecast', color='red')
plt.title(f"Sales Forecast for next {future_days} days")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.show()

